In [18]:
import math
import time
import random
import copy

## Problema voos

In [19]:
pessoas = [('Amanda', 'CWB'),
           ('Pedro', 'GIG'),
           ('Marcos', 'POA'),
           ('Priscila', 'FLN'),
           ('Jessica', 'CNF'),
           ('Paulo', 'GYN')]

destino = 'GRU'

In [20]:
voos = {}

for l in open('voos.txt'):
    origem, dest, saida, chegada, preco = l.split(',')
    voos.setdefault((origem, dest), [])
    voos[(origem, dest)].append((saida, chegada, int(preco)))

In [21]:
def print_agenda(agenda):
    for idx, voo in enumerate(agenda):
        nome = pessoas[idx][0]
        origem = pessoas[idx][1]

        ida = voos[(origem, destino)][voo[0]]
        volta = voos[(destino, origem)][voo[1]]

        print("%10s%10s %5s-%5s R$%3s %5s-%5s R$%3s" % (nome, origem, ida[0], ida[1], ida[2],
                                                        volta[0], volta[1], volta[2]))

agenda = [[1,3], [3, 2], [7,3], [6,3], [2,4], [5,3]]

print_agenda(agenda)

    Amanda       CWB  8:04-10:11 R$ 95 10:33-12:03 R$ 74
     Pedro       GIG 10:30-14:57 R$290  9:49-13:51 R$229
    Marcos       POA 17:08-19:08 R$262 10:32-13:16 R$139
  Priscila       FLN 15:34-18:11 R$326 11:08-14:38 R$262
   Jessica       CNF  9:42-11:32 R$169 12:08-14:47 R$231
     Paulo       GYN 13:37-15:08 R$250 11:07-13:24 R$171


In [22]:
def get_minutos(hora:str):
    tempo = time.strptime(hora, '%H:%M')
    return tempo[3] * 60 + tempo[4]

get_minutos('1:10')

70

In [23]:
def custo(solucao:list):
    preco_total = 0
    # menor possível
    ultima_chegada = 0
    # maior possível
    primeira_partida = 1439 #23:59 em minutos
    total_espera = 0

    for idx, voo in enumerate(solucao):
        origem = pessoas[idx][1]

        ida = voos[(origem, destino)][voo[0]]
        volta = voos[(destino, origem)][voo[1]]

        preco_total += ida[2]
        preco_total += volta[2]

        # atualiza a ultima chegada ao aeroporto
        if ultima_chegada < get_minutos(ida[1]):
            ultima_chegada = get_minutos(ida[1])

        # atualiza com a primeira chegada ao aeroporto
        if primeira_partida > get_minutos(volta[0]):
            primeira_partida = get_minutos(volta[0])
        
    for idx, voo in enumerate(solucao):
        origem = pessoas[idx][1]

        ida = voos[(origem, destino)][voo[0]]
        volta = voos[(destino, origem)][voo[1]]

        # calcula o tanto de espera no aeroporto
        total_espera += ultima_chegada - get_minutos(ida[1])
        total_espera += get_minutos(volta[0]) - primeira_partida

    # se virou um dia, adiciona penalidade em dinheiro
    if ultima_chegada > primeira_partida:
        preco_total += 50

    return preco_total + total_espera

In [24]:
custo(agenda)

4472

## Busca randômica

In [25]:
def busca_randomica(dominio, funcao_custo, numero_solucao):
    melhor_custo = 99999999
    melhor_solucao = None

    for i in range(numero_solucao):
        solucao = [[random.randint(dominio[i][0], dominio[i][1]), random.randint(dominio[i][0], dominio[i][1])] for i in range(len(dominio))]

        custo = funcao_custo(solucao)

        if custo < melhor_custo:
            melhor_custo = custo
            melhor_solucao = solucao

    return melhor_solucao

# para cada pessoa, tem-se o numero possivel de voos.
dominio = [[0, 9]] * (len(pessoas))

busca_randomica(dominio, funcao_custo=custo, numero_solucao=1000)

[[0, 3], [0, 2], [3, 5], [2, 5], [2, 2], [1, 7]]

## Hill Climb

In [26]:
def hill_climb(domino, funcao_custo):
    # gera uma solucao randomica para iniciar o algoritmo
    solucao = busca_randomica(dominio=dominio, funcao_custo=funcao_custo,  numero_solucao=1000)

    print('Solucao aleatoria: ', solucao)

    while True:
        vizinhos = []
    
        for i in range(len(dominio)):
            # le cada valor da tupla
            for j in range(len(dominio[i])):
                 # verifica se o valor é menor que o valor maximo do dominio
                if solucao[i][j] < dominio[i][1]:
                    # faz uma copida da lista a ser modificado
                    novo_vizinho = copy.deepcopy(solucao)
                    # modifica o valor da solucao na posica [i][j]
                    novo_vizinho[i][j] = novo_vizinho[i][j] + 1
                    # adiciona a lista modificada aos vizinhos
                    vizinhos.append(novo_vizinho)
            
                # verifica se o valor é maior que o valor minimo do dominio
                if solucao[i][j] > dominio[i][0]:
                    # faz uma copida da lista a ser modificado
                    novo_vizinho = copy.deepcopy(solucao)
                    # modifica o valor da solucao na posica [i][j]
                    novo_vizinho[i][j] = novo_vizinho[i][j] - 1
                    # adiciona a lista modificada aos vizinhos
                    vizinhos.append(novo_vizinho)
        # verifica o custo da solucao aleatorea
        atual = funcao_custo(solucao)
        # adiciona o custo da funcao aleatórea a melhor
        melhor = atual

        for i in range(len(vizinhos)):
            # print(vizinhos[i])
            custo = funcao_custo(vizinhos[i])

            if custo < melhor:
                # atualiza para o melhor custo
                melhor = custo
                # registra a melhor solucao
                solucao = vizinhos[i]

        # melhro solucao encontrada
        if melhor == atual:
            break

    print('Melhor solucão: ', solucao)
    print('Melhor custo: ', melhor)
    print_agenda(agenda=solucao)
    
    return solucao

In [27]:
hill_climb(domino=dominio, funcao_custo=custo)

Solucao aleatoria:  [[5, 9], [3, 5], [5, 6], [2, 8], [0, 5], [2, 8]]
Melhor solucão:  [[5, 6], [3, 5], [5, 6], [3, 6], [2, 5], [2, 6]]
Melhor custo:  3068
    Amanda       CWB 13:40-15:37 R$138 15:25-16:58 R$ 62
     Pedro       GIG 10:30-14:57 R$290 14:20-17:32 R$332
    Marcos       POA 13:40-15:38 R$137 15:50-18:45 R$243
  Priscila       FLN 11:28-14:40 R$248 15:23-18:49 R$150
   Jessica       CNF  9:42-11:32 R$169 14:19-17:09 R$190
     Paulo       GYN  9:15-12:03 R$ 99 15:07-17:21 R$129


[[5, 6], [3, 5], [5, 6], [3, 6], [2, 5], [2, 6]]

## Simulated Annealing

In [28]:
def simulated_annealing(dominio, funcao_custo, temperatura = 10000.0, taxa_resfriamento = 0.95, passo = 1):
    # pega uma solucao randomica para inicializar o algoritmo
    solucao = busca_randomica(dominio=dominio, funcao_custo=funcao_custo,  numero_solucao=1000)

    while temperatura > 0.1:
        # escolhe uma posicao aleatoria para fazer a modificacao
        random_idx = random.randint(0, len(dominio) - 1)
        random_idx_dupla = random.randint(0, 1)

        # escolhe a direcao a que quer mudar. Caso passo = 1, posicao randomica entre -1 e 1
        direcao = random.randint(-passo, passo) 

        solucao_mod = copy.deepcopy(solucao)
        # modifica a posicao aleatoria gerada
        solucao_mod[random_idx][random_idx_dupla] += direcao

        # se for negativo, coloca 0
        if solucao_mod[random_idx][random_idx_dupla] < dominio[random_idx][0]:
            solucao_mod[random_idx][random_idx_dupla] = dominio[random_idx][0]
        # se for maior que o max do dominio, coloca o max
        elif solucao_mod[random_idx][random_idx_dupla] > dominio[random_idx][1]:
            solucao_mod[random_idx][random_idx_dupla] = dominio[random_idx][1]

        # calcula os custos
        custo_solucao = funcao_custo(solucao)
        custo_solucao_mod = funcao_custo(solucao_mod)
        
        # calcula a probabilidade de escolher uma solucao 'pior'
        prob = pow(math.e, (-custo_solucao_mod - custo_solucao) / temperatura)

        if custo_solucao_mod < custo_solucao or random.random() < prob:
            solucao = solucao_mod

        # diminui a temperatura
        temperatura *= taxa_resfriamento

    print('Melhor solucão: ', solucao)
    print('Melhor custo: ', custo_solucao)
    print_agenda(agenda=solucao)

    return solucao

In [29]:
simulated_annealing(dominio=dominio, funcao_custo=custo)

Melhor solucão:  [[7, 3], [3, 2], [6, 5], [6, 1], [6, 1], [6, 1]]
Melhor custo:  3333
    Amanda       CWB 17:11-18:30 R$108 10:33-12:03 R$ 74
     Pedro       GIG 10:30-14:57 R$290  9:49-13:51 R$229
    Marcos       POA 15:23-17:25 R$232 13:37-15:33 R$142
  Priscila       FLN 15:34-18:11 R$326  8:23-11:07 R$143
   Jessica       CNF 15:58-18:40 R$173  7:50-10:08 R$164
     Paulo       GYN 15:03-16:42 R$135  8:04-10:59 R$136


[[7, 3], [3, 2], [6, 5], [6, 1], [6, 1], [6, 1]]

## Algoritmo Genético

In [30]:
def mutacao(dominio, passo, solucao):
    # pega um valor aleatorio para realizar a mutacao
    random_idx = random.randint(0, len(solucao) -1)
    random_idx_dupla = random.randint(0, 1)

    # copia o valor da solucao para a alteracao
    mutante = copy.deepcopy(solucao)

    # probabilidade de aumentar ou diminuir o valor da mutacao
    if random.random() < 0.5:
        # verifica se é diferente de 0
        if solucao[random_idx][random_idx_dupla] != dominio[random_idx][random_idx_dupla]:
            mutante[random_idx][random_idx_dupla] -= passo
    else:
        # verifica se é diferente de 9
        if solucao[random_idx][random_idx_dupla] != dominio[random_idx][random_idx_dupla]:
            mutante[random_idx][random_idx_dupla] += passo

    return mutante

mutacao(dominio=dominio, passo=1, solucao=[[8, 2], [8, 2], [8, 1], [9, 2], [9, 1], [8, 3]])


[[8, 2], [8, 2], [8, 2], [9, 2], [9, 1], [8, 3]]

In [31]:
def cruzamento(dominio, individuo1, individuo2):
    ponto_corte = random.randint(1, len(dominio) - 1)
    return individuo1[0:ponto_corte] + individuo2[ponto_corte:]

cruzamento(dominio=dominio,
           individuo1=[[2, 3], [0, 2], [1, 5], [2, 6], [2, 3], [2, 3]],
           individuo2=[[5, 3], [3, 5], [5, 3], [3, 2], [2, 3], [4, 2]])

[[2, 3], [0, 2], [1, 5], [2, 6], [2, 3], [4, 2]]

In [32]:
def genetico(dominio, funcao_custo, tamanho_populacao = 50, passo = 1, prob_mutacao = 0.2,
             elitismo = 0.2, numero_geracao = 100):
    # gera um numero de populacao aleatoria
    populacao = []
    for i in range(tamanho_populacao):
        populacao.append(busca_randomica(dominio=dominio, funcao_custo=funcao_custo,  numero_solucao=1000))

    # numero de individuos que serao selecionados como pais
    numero_etilitismo = int(elitismo * tamanho_populacao)

    for _ in range(numero_geracao):
        # cria uma lista de tuplas com o individuo e seu custo
        # para conseguir realizar a ordenacao
        custos = [(funcao_custo(individuo), individuo) for individuo in populacao]
        custos.sort()
        
        # pega a relacao de individuos ordenados
        individuos_ordenados = [individuo for (_, individuo) in custos]

        # pega só os melhores individuos
        populacao = individuos_ordenados[0:numero_etilitismo]

        # irá criar mais individuos para compor a populacao
        while len(populacao) < tamanho_populacao:
            # testa a probabilidade de mutacao
            if random.random() < prob_mutacao:
                # seleciona um idx de um individuo aleatorio
                mutacao_idx = random.randint(0, numero_etilitismo)
                # adiciona o individuo mutado a populacao
                populacao.append(mutacao(dominio=dominio, passo=passo, solucao=individuos_ordenados[mutacao_idx]))
            # se nao, realiza o cruzamento entre dois individuos diferentes
            else:
                ind1 = random.randint(0, numero_etilitismo)
                ind2 = random.randint(0, numero_etilitismo)

                # verifica se o ind1 é diferente do ind2
                if ind1 == ind2:
                    # garante que o ind1 vai ser diferente q o ind2
                    while ind1 == ind2:
                        ind2 = random.randint(0, numero_etilitismo)

    print('Melhor solucao: ', custos[0][1])
    print('Melhor custo: ', custos[0][0])
    print_agenda(agenda=custos[0][1])
                
    return custos[0]

In [33]:
genetico(dominio=dominio, funcao_custo=custo)

Melhor solucao:  [[2, 6], [0, 7], [1, 6], [2, 6], [2, 6], [2, 6]]
Melhor custo:  2656
    Amanda       CWB  9:45-11:50 R$172 15:25-16:58 R$ 62
     Pedro       GIG  6:12-10:22 R$230 17:14-20:59 R$277
    Marcos       POA  8:27-10:45 R$139 15:50-18:45 R$243
  Priscila       FLN  9:15-12:29 R$225 15:23-18:49 R$150
   Jessica       CNF  9:42-11:32 R$169 15:04-17:23 R$189
     Paulo       GYN  9:15-12:03 R$ 99 15:07-17:21 R$129


(2656, [[2, 6], [0, 7], [1, 6], [2, 6], [2, 6], [2, 6]])

## Problema Dormitórios

In [34]:
dormitorios = ['sao paulo', 'flamengo', 'coritiba', 'cruzeiro', 'fortaleza']

preferencias = {
    'amanda' : ('cruzeiro', 'coritiba'),
    'pedro' : ('sao paulo', 'fortaleza'),
    'marcos' : ('flamengo', 'sao paulo'),
    'priscila' : ('sao paulo', 'fortaleza'),
    'jessica' : ('flamengo', 'cruzeiro'),
    'paulo' : ('coritiba', 'fortaleza'),
    'fred' : ('fortaleza', 'flamengo'),
    'suzana' : ('cruzeiro', 'coritiba'),
    'laura' : ('cruzeiro', 'coritiba'),
    'ricardo' : ('coritiba', 'flamengo')
}

In [35]:
dominio = [(0, (len(dormitorios) * 2) - i - 1) for i in range(len(dormitorios) * 2)]
dominio

[(0, 9),
 (0, 8),
 (0, 7),
 (0, 6),
 (0, 5),
 (0, 4),
 (0, 3),
 (0, 2),
 (0, 1),
 (0, 0)]

In [44]:
def print_vagas(solucao:list):
    vagas = []

    for i in range(len(dormitorios)):
        # deminstrar que para cada quarto haverá duas vagas
        # ex: 0, 0, 1, 1. Indo pelo indice dos dormitorios
        vagas += [i, i]

    for i in range(len(solucao)):
        # pega o numero do indice do dormitorio
        dormitorio = dormitorios[vagas[solucao[i]]]
        print(list(preferencias.keys())[i], dormitorio)
        # apos preencher a vaga, retira ela
        # limpa a lista de vagas
        del vagas[solucao[i]]

print_vagas(solucao=[6,1,2,1,2,0,2,2,0,0])

amanda cruzeiro
pedro sao paulo
marcos flamengo
priscila flamengo
jessica coritiba
paulo sao paulo
fred fortaleza
suzana fortaleza
laura coritiba
ricardo cruzeiro


In [52]:
def custo_dormitorio(solucao:list):
    vagas = []
    custo = 0

    for i in range(len(dormitorios)):
        # deminstrar que para cada quarto haverá duas vagas
        # ex: 0, 0, 1, 1. Indo pelo indice dos dormitorios
        vagas += [i, i]

    for i in range(len(solucao)):
        # pega o dormitorio na posicao i
        dormitorio = dormitorios[vagas[solucao[i]]]
        # recupera as preferencias da pessoa atual na posicao i
        pref = preferencias.get(list(preferencias.keys())[i])
        
        # primeiro quarto desejado
        if dormitorio == pref[0]:
            pass
        # segunda preferencia + 1 pen
        elif dormitorio == pref[1]:
            custo += 1
        # nenhuma das duas + 3 pen
        else:
            custo += 3

        del vagas[solucao[i]]

    return custo


custo_dormitorio(solucao=[6,1,2,1,2,0,2,2,0,0])

16